In [2]:
from music21 import *
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_rows = 999
# Listing current data on our folder.
import os
print(os.listdir("."))
from music21 import converter, corpus, instrument, midi, note, chord, pitch, roman

import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
# Building a english to french translator

import keras
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
from keras.models import model_from_json
from keras.callbacks import EarlyStopping, ModelCheckpoint
from importlib import reload
reload(keras.models)

import pickle
import random
import h5py

#https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
#https://github.com/devm2024/nmt_keras/blob/master/base.ipynb


['including_rests_clean-Copy1.ipynb', '.DS_Store', 'get_music.ipynb', 'all_4.py', 'dict.pkl', 'processed_songs.pkl', '.ipynb_checkpoints', 'run_model.ipynb', 'get_data-Copy1.ipynb']


In [3]:
import pickle
with open('processed_songs.pkl', 'rb') as picklefile:
    processed_songs = pickle.load(picklefile)

In [4]:
lines = pd.concat(processed_songs).reset_index()

Thanks Devesh Maheshwari for your part in this code!
https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7

In [5]:
lines['harmony'] = lines['harmony'].str.replace('null','rest')
lines['harmony'] = lines['harmony'].apply(lambda x : 'START_ '+ x + ' _END')

## Encoding Target

In [6]:
# Create vocabulary of chords
all_chords=set()
for harmony in lines.harmony:
    for ch in harmony.split():
        if ch not in all_chords:
            all_chords.add(ch)
            
target_chords = sorted(list(all_chords))
num_decoder_tokens = len(all_chords)            
            

In [7]:
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_chords)])

In [8]:
# max number of chords in a measure.
lenght_list=[]
for l in lines.harmony:
    lenght_list.append(len(l.split(' ')))
    
# try this length_list = [len(x.split(' ')) for x in lines.harmony]
target_max = np.max(lenght_list)

In [9]:
decoder_input_data = np.zeros(
    (len(lines.harmony), target_max),
    dtype='float32')
# Shape: total number of measures, chords per measure.

decoder_target_data = np.zeros(
    (len(lines.harmony), target_max, num_decoder_tokens),
    dtype='float32')
# Shape: total number of measures, chords per measure, unique chords.

In [10]:
for i, target_chords in enumerate(lines.harmony):
    for t, ch in enumerate(target_chords.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[ch]
        if t > 0:
            # decoder_target_data will be ahead by one timestep target_token_index[ch]
            # and will not include the start character.
            decoder_target_data[i, t - 1,target_token_index[ch]] = 1.

## Encoding input data

In [11]:
# Encoding input data
input_max = 16 # Max number of notes per measure
# input_max = [len(x) for x in lines.harmony]
# input_max = np.max(input_max)
num_encoder_tokens = len(lines['melody'].values[0][0]) # 23 = number of unique notes + rest.

zeros = [0]*num_encoder_tokens
input_data = lines['melody'].values

# Unifying the shapes of the measures to create a 3d array. 
for measure_number in range(len(input_data)):
    while len(input_data[measure_number]) < input_max:
        input_data[measure_number] = np.append(input_data[measure_number],np.array([zeros]),axis=0)
    if len(input_data[measure_number]) > input_max:
        input_data[measure_number] = input_data[measure_number][:input_max]


input_data = np.array(list(input_data))
encoder_input_data = input_data.reshape((len(input_data),input_max,num_encoder_tokens))


### Model architecture

In [12]:
# Define input sequence
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
embedding_size=50

In [14]:
# Set up the decoder, using `encoder_states` as initial state. (None, num_decoder_tokens?)
decoder_inputs = Input(shape=(None,))

dex = Embedding(num_decoder_tokens, embedding_size) # num_decoder_tokens = number of unique chords.
final_dex= dex(decoder_inputs)

decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder inputs and decoder inputs into decoder outputs.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Model training

In [15]:
# optimizer; adam rmsprop
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None, 23)     0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     5200        input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 50), (None,  14800       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

In [19]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5),ModelCheckpoint(filepath='best_model_farm.h5', monitor='val_loss', save_best_only=True)]

In [20]:
# Batch size = 512?
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=50,
          validation_split=0.20, callbacks = callbacks
         )

Train on 257 samples, validate on 65 samples
Epoch 1/50
257/257 [==============================] - 0s 833us/step - loss: 1.3791 - acc: 0.0707 - val_loss: 2.5347 - val_acc: 0.0615
Epoch 2/50
257/257 [==============================] - 0s 693us/step - loss: 1.3814 - acc: 0.0683 - val_loss: 2.5537 - val_acc: 0.0598
Epoch 3/50
257/257 [==============================] - 0s 725us/step - loss: 1.3734 - acc: 0.0750 - val_loss: 2.5874 - val_acc: 0.0581
Epoch 4/50
257/257 [==============================] - 0s 626us/step - loss: 1.3780 - acc: 0.0737 - val_loss: 2.5915 - val_acc: 0.0607
Epoch 5/50
257/257 [==============================] - 0s 692us/step - loss: 1.3757 - acc: 0.0741 - val_loss: 2.5478 - val_acc: 0.0607
Epoch 6/50
257/257 [==============================] - 0s 616us/step - loss: 1.3629 - acc: 0.0770 - val_loss: 2.5050 - val_acc: 0.0581
Epoch 7/50
257/257 [==============================] - 0s 588us/step - loss: 1.3645 - acc: 0.0778 - val_loss: 2.5112 - val_acc: 0.0607
Epoch 8/50
257/25

In [373]:
# model_json = model.to_json()
# with open("models/final_test/model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("models/final_test/model.h5")
# print("Saved model to disk")

Saved model to disk


In [21]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 23)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  14800     
Total params: 14,800
Trainable params: 14,800
Non-trainable params: 0
_________________________________________________________________


In [22]:

decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.

reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [23]:
# serialize model to JSON
model_json = encoder_model.to_json()
with open("encoder_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
encoder_model.save_weights("encoder_model.h5")
print("Saved model to disk")

# serialize model to JSON
model_json = decoder_model.to_json()
with open("decoder_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
decoder_model.save_weights("decoder_model.h5")
print("Saved model to disk")



with open('target_token_index.pkl', 'wb') as picklefile:
    pickle.dump(target_token_index, picklefile)


with open('reverse_target_char_index.pkl', 'wb') as picklefile:
    pickle.dump(reverse_target_char_index ,picklefile)


Saved model to disk
Saved model to disk


/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_3:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'input_4:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
